In [2]:
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.train_test import *
from src.data_prep import *
import pandas as pd
import pymc3 as pm

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [3]:
data_dir = "/Users/samrelins/Documents/LIDA/ace_project/data/"
orig_filename = "ace_data_orig.csv"
ace_dat_orig = pd.read_csv(data_dir + orig_filename)

ace_dat_orig.head()

,hospital_reqd,referral_from,age,address,ethnicity,gender,allergies,referral_date,referral_time,illness_severity,activity_level,gut_feeling,ox_sat,resp_rate,heart_rate,temp,sepsis,safeguarding
0,N,CCDA,8,BD07,Indian,F,NKA,Winter,Morning,Moderate,usual,low concern,97,20,118,36.5,None noted,N
1,N,A&E,11,BD03,Pakistani,F,NKA,Winter,Morning,Mild,lower,low concern,96,20,109,37,None noted,N
2,N,CCDA,3,BD04,Slovak,F,NKDA,Winter,Afternoon,Mild,usual,well,96,28,140,37,None noted,N
3,N,GP,3,BD06,British,M,NKDA,Winter,Afternoon,Mild,usual,low concern,98,28,104,36.8,None noted,N
4,N,GP,3,BD09,Pakistani,M,NKA,Winter,Afternoon,Mild,usual,well,97,None,None,None,None noted,N


In [4]:
X_train, y_train, X_test, y_test = return_train_test(
    ace_dat_orig, run_pipeline=True
)
X_train, X_test = encode_and_scale(X_train, y_train, X_test,
                                   cat_encoder="one_hot",
                                   scaled=True)
X_train.columns

Index(['referral_from_ccda', 'referral_from_ed', 'referral_from_gp',
       'address_bd01', 'address_bd02', 'address_bd03', 'address_bd04',
       'address_bd05', 'address_bd06', 'address_bd07', 'address_bd08',
       'address_bd09', 'address_bd10', 'address_bd11', 'address_bd12',
       'address_bd13', 'address_bd14', 'address_bd15', 'address_bd16',
       'address_bd17', 'address_bd18', 'address_bd19', 'address_ls20',
       'address_ls29', 'gender_m', 'referral_date_autumn',
       'referral_date_spring', 'referral_date_summer', 'referral_date_winter',
       'referral_time_afternoon', 'referral_time_evening',
       'referral_time_morning', 'illness_severity_moderate',
       'activity_level_usual', 'gut_feeling_low concern', 'gut_feeling_unwell',
       'gut_feeling_well', 'sepsis_none noted', 'safeguarding_y',
       'food_allergy_y', 'drug_allergy_y', 'other_allergy_y',
       'simple_ethnicity_other', 'group_ethnicity_other', 'ox_sat_low_y',
       'age_range_pre_school', 'age_

### simple model to test basics and write helper functions to visualise output:

In [ ]:
simple_model_features = [
    'referral_from_ccda', 'referral_from_ed', 'referral_from_gp',
    'gender_m', 'referral_date_autumn', 'illness_severity_moderate',
    'activity_level_usual', 'gut_feeling_low concern', 
    'gut_feeling_unwell', 'gut_feeling_well',  'food_allergy_y', 
    'drug_allergy_y', 'other_allergy_y', 'simple_ethnicity_other', 
    'group_ethnicity_other',  'age', 'ox_sat', 'resp_rate', 
    'heart_rate', 'temp'
]
with pm.Model() as log_reg_model:
    pm.glm.GLM(
        x = X_train[simple_model_features],
        y = y_train,
        intercept=True,
        family=pm.glm.families.Binomial()
    )
    trace = pm.sample(1000, 
                      tune=3000, 
                      cores=6,
                      target_accept=.9,
                      init="adapt_diag")


Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (6 chains in 6 jobs)
NUTS: [temp, heart_rate, resp_rate, ox_sat, age, group_ethnicity_other, simple_ethnicity_other, other_allergy_y, drug_allergy_y, food_allergy_y, gut_feeling_well, gut_feeling_unwell, gut_feeling_low concern, activity_level_usual, illness_severity_moderate, referral_date_autumn, gender_m, referral_from_gp, referral_from_ed, referral_from_ccda, Intercept]


In [21]:
import plotly.express as px
fig = px.histogram(df, x=trace["gut_feeling_well"])
fig.show()

In [139]:
all_coef_samples = trace["Intercept"].reshape(1,-1)
for feature in keep_features:
    feature_samples = trace[feature].reshape(1,-1)
    all_coef_samples = np.concatenate([all_coef_samples, feature_samples])
    

In [143]:
all_coef_samples.shape

(9, 6000)

In [141]:
np.concatenate([[1], (X_train[keep_features].sample(1).values).reshape(-1,)]).shape

(9,)

In [222]:
def plot_preds():
    x_vec = X_train[keep_features].sample(1).values
    x_vec_plus_int = np.concatenate([[1], x_vec.reshape(-1,)])
    z_vec = x_vec_plus_int @ all_coef_samples
    log_vec = list(1 / (1 + np.exp(-z_vec)))
    print(np.mean(log_vec))
    fig = px.histogram(x=log_vec)
    fig.show()

plot_preds()

0.30829360641390263


In [82]:
X_train[keep_features].sample(1).values

array([[0.        , 0.21428571, 0.17647059, 0.46226415, 0.6       ,
        0.27027027, 1.        , 1.        ]])

In [111]:
trace.varnames

['Intercept',
 'referral_from_gp',
 'age',
 'resp_rate',
 'heart_rate',
 'ox_sat',
 'temp',
 'gut_feeling_well',
 'activity_level_usual']